# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [2]:
import pandas as pd
import numpy as np
import math

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

In [175]:
train = pd.read_csv("wikipedia_train3.csv")
test = pd.read_csv("wikipedia_test3.csv")
train.loc[:,'date'] = pd.to_datetime(train['date'])
test.loc[:,'date'] = pd.to_datetime(test['date'])

In [208]:
delayLength = (test.iloc[0, 1] - train.iloc[-1, 1])
testLength = (test.iloc[-1, 1] - test.iloc[0, 1])
trainEnd = train.iloc[-1, 1]

train_train = train[train['date'] <= trainEnd - testLength*0.62 -  delayLength]
train_test = train[train['date'] >= trainEnd - testLength*0.62 ] 

In [2]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [3]:
def medianPred(X , Y , start_date ):
    median_dayOfWeek = X[X["date"] >= start_date ].groupby(['Page']).median().reset_index()
    median_dayOfWeek.rename(columns={"Visits": "pred_Visits"}, inplace=True)
    pred =Y.merge(median_dayOfWeek, on=["Page"], how='left')
    return smape_fast(pred['Visits'] , pred['pred_Visits'])

In [155]:
val_score = medianPred(train_train , train_test , train_train['date'].unique()[-15] )
test_score = medianPred(train , test , train['date'].unique()[-15] )
dif = abs(val_score - test_score)
print('val_score : ' + str(val_score))
print('test_score : ' + str(test_score))
print('dif : ' + str(dif))

val_score : 39.99415886279752
test_score : 40.00701270204617
dif : 0.012853839248649024


---

In [157]:
val_score = medianPred(train_train , train_test , train_train['date'].unique()[-10] )
test_score = medianPred(train , test , train['date'].unique()[-10] )
dif = abs(val_score - test_score)
print('val_score : ' + str(val_score))
print('test_score : ' + str(test_score))
print('dif : ' + str(dif))

val_score : 39.382484866455705
test_score : 39.50827415010651
dif : 0.12578928365080344


In [159]:
val_score = medianPred(train_train , train_test , train_train['date'].unique()[-20] )
test_score = medianPred(train , test , train['date'].unique()[-20] )
dif = abs(val_score - test_score)
print('val_score : ' + str(val_score))
print('test_score : ' + str(test_score))
print('dif : ' + str(dif))

val_score : 40.69365661456377
test_score : 40.8186411546377
dif : 0.12498454007393178


In [154]:
val_score = medianPred(train_train , train_test , train_train['date'].unique()[-30] )
test_score = medianPred(train , test , train['date'].unique()[-30] )
dif = abs(val_score - test_score)
print('val_score : ' + str(val_score))
print('test_score : ' + str(test_score))
print('dif : ' + str(dif))

val_score : 41.565609914551246
test_score : 41.480911818504154
dif : 0.08469809604709155


**3.** Perform K-fold validation using your type of split. Run GridSearch with any classificator you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

In [4]:
train = pd.read_csv("wikipedia_train3.csv")
test = pd.read_csv("wikipedia_test3.csv")

train.loc[:,'date'] = pd.to_datetime(train['date'])
test.loc[:,'date'] = pd.to_datetime(test['date'])

test['dayOfWeek'] = [ el.dayofweek for el in  test['date'] ]
train['dayOfWeek'] = [ el.dayofweek for el in  train['date'] ]

In [5]:
uPage = train['Page'].unique()
intPage = pd.Series( range(uPage.size) , index= uPage )
intPage = intPage.reset_index()
intPage = intPage.rename(columns = {0 : 'id_Page' , 'index': 'Page'} )
intPage.head()

,Page,id_Page
0,Special:Search_zh.wikipedia.org_all-access_spider,0
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,1
2,Project:上传_zh.wikipedia.org_all-access_spider,2
3,Special:用户登录_zh.wikipedia.org_all-access_spider,3
4,Championnat_d'Europe_de_football_2016_fr.wikip...,4


In [6]:
train = train.merge( intPage , on = 'Page' , how = 'left').drop('Page', axis =1)
test = test.merge( intPage , on = 'Page' , how = 'left').drop('Page', axis = 1)

In [7]:
train.head()

,date,Visits,dayOfWeek,id_Page
0,2016-01-01,2660.0,4,0
1,2016-01-01,9936.0,4,1
2,2016-01-01,1712.0,4,2
3,2016-01-01,3175.0,4,3
4,2016-01-01,1682.0,4,4


In [8]:
delayLength = (test.iloc[0, 0] - train.iloc[-1, 0])
testLength = (test.iloc[-1, 0] - test.iloc[0, 0])
trainEnd = train.iloc[-1, 0]

train_train = train[train['date'] <= trainEnd - testLength -  delayLength]
train_test = train[train['date'] >= trainEnd - testLength ] 

In [9]:
from dateutil.relativedelta import relativedelta
def create_validation(df, start_date):
    tmpDf_train = df[   (start_date <=df['date']) &
                        (df['date'] <= start_date + relativedelta(days=10)) ]
                         
    tmpDf_test = df[    (df['date'] > start_date + relativedelta(days=10) + delayLength)&
                        (df['date'] < start_date + relativedelta(days=10) + delayLength + testLength) ]
    
    return tmpDf_train.index , tmpDf_test.index

In [10]:
uDate = pd.to_datetime(train_train.date.unique())
date_to_use =uDate[uDate < trainEnd - (relativedelta(days=10) + delayLength + testLength)]
k =1
train_dates = [ date_to_use[int(i*date_to_use.size/k)] for i in range(k) ]
train_dates

[Timestamp('2016-01-01 00:00:00')]

In [11]:
### Your validation iterator
CVIterator = []
for date in train_dates:
    train_ , test_ = create_validation(train_train.reset_index().drop('index',axis= 1) , date)
    CVIterator.append(( train_ , test_))
for x, y in CVIterator:
    print(min(x) , max(x) , min(y) , max(y) )

0 129964 248115 957014


In [12]:
def smape(estimator, X, y):
    return(smape_fast(y, estimator.predict(X)))

In [13]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def mape_GS( estimator , X , y):
    return mape(estimator.predict(x) , y)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
treeRegressor = DecisionTreeRegressor()

params = {'max_depth': [ None] }

gs = GridSearchCV(
    estimator=treeRegressor,
    param_grid=params, 
    cv=CVIterator, 
    error_score=1, 
    #scoring= smape , 
    n_jobs=-1,
    verbose=2,  
)

gs.fit(X=train_train.drop(['date','Visits'] , axis = 1) ,y= train_train['Visits'])
best_score = gs.best_score_
best_dt = gs.best_estimator_

Fitting 1 folds for each of 1 candidates, totalling 1 fits


In [ ]:
best_score

In [ ]:
best_dt

In [ ]:
y_test_pred =  gs.predict(train_test.drop(['Visits', 'date'  ], axis =1 ) )
smape_fast(train_test['Visits'], y_test_pred)

In [ ]:
test_pred =  gs.predict(test.drop(['Visits', 'date'  ], axis =1 ) )
smape_fast(test['Visits'], test_pred)

In [14]:
from sklearn.tree import DecisionTreeRegressor
treeRegressor = DecisionTreeRegressor()
treeRegressor.fit(train_train.drop(['Visits', 'date'], axis =1 ) ,train_train['Visits'] )
y_test_pred =  treeRegressor.predict(train_test.drop(['Visits', 'date'  ], axis =1 ) )
smape_fast(train_test['Visits'], y_test_pred)

63.1614835090445

In [15]:
treeRegressor.fit(train.drop(['Visits', 'date'], axis =1 ) ,train['Visits'] )
test_pred =  treeRegressor.predict(test.drop(['Visits', 'date'  ], axis =1 ) )
smape_fast(test['Visits'], test_pred)

59.04121904290289

## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [3]:
train = pd.read_csv('train_french.csv')
test = pd.read_csv('test_french.csv')

In [4]:
import pandas_profiling as pp

In [167]:
pp.ProfileReport(train)

Number of variables,29
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,44.3 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


In [168]:
pp.ProfileReport(test)

Number of variables,29
Number of observations,75968
Total Missing (%),0.0%
Total size in memory,16.8 MiB
Average record size in memory,232.0 B
Numeric,15
Categorical,6
Boolean,8
Date,0
Text (Unique),0
Rejected,0


In [5]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat( [df[cat_cols] for df in df_list] , axis=0 )\
            .fillna('')\
            .apply(lambda x: d[x.name].fit(x)),
            
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(lambda x: d[x.name].transform(x))

In [6]:
featureList = train.columns.drop('prime_tot_ttc')
uniqueValues = pd.Series([train[col].value_counts().size for col in featureList ] , index = featureList)

binary_cols = list(uniqueValues.index[uniqueValues == 2])
categorical_cols = ['nature_lri' ,"niv_urbanisation", 'profession', 'code_postal']
ordinal_cols = list(uniqueValues.index.drop((binary_cols + categorical_cols)))

__encode_categorical(df_list= [train, test], cat_cols= categorical_cols + binary_cols )

In [7]:
train.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,16131,1,0,500,1,1,5,1,34,0,...,0,194,49,0,0,1,0,30,1,67.68
1,22013,2,0,500,1,4,5,1,37,0,...,0,0,100,0,0,5,0,12,0,120.60
2,18092,3,0,500,1,10,2,0,1,4,...,1,98,100,0,0,5,0,30,1,100.62
3,7493,3,0,500,0,1,2,1,34,0,...,0,151,38,0,0,1,0,15,0,58.73
4,19264,2,0,500,0,9,0,0,19,3,...,0,39,100,0,0,1,0,0,1,132.65


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [34]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('prime_tot_ttc', axis = 1), 
                                                    train['prime_tot_ttc'],
                                                    test_size= test.shape[0]/(train.shape[0] + test.shape[0]),
                                                    random_state=100,                                            )

In [35]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [36]:
treeRegressor = DecisionTreeRegressor()
treeRegressor.fit( X_train ,  y_train )
mape( y_test, treeRegressor.predict(X_test))

11.40617632713923

In [37]:
treeRegressor = DecisionTreeRegressor()
treeRegressor.fit( train.drop('prime_tot_ttc',axis = 1) ,  train['prime_tot_ttc'] )
mape( test['prime_tot_ttc'], treeRegressor.predict(test.drop('prime_tot_ttc', axis =1)))

10.919879226593913